<a href="https://colab.research.google.com/github/alecinvan/Colab_AI_tutorial/blob/main/Colab_Stable_Diffusion_tryout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<font color="blue" size="8">使用Stable Diffusion构建一个艺术创作空间



#<font color="darkred" size="5">这里我们使用huggingface上的开源stable-diffusion模型：
https://huggingface.co/CompVis/stable-diffusion-v1-4

Endpoints: API_URL = "https://api-inference.huggingface.co/models/CompVis/stable-diffusion-v1-4"


In [ ]:
%pip install --quiet --upgrade diffusers transformers scipy mediapy accelerate

In [ ]:
!pip install gradio -q

In [ ]:
!huggingface-cli login

Now we set up our model and pipeline. IF you want to remove the NSFW safety checker, change "remove_safety" to "True".

In [ ]:
from diffusers import PNDMScheduler, DDIMScheduler, LMSDiscreteScheduler

scheduler = PNDMScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", skip_prk_steps=True)

In [ ]:
import mediapy as media
import torch
from torch import autocast
from diffusers import StableDiffusionPipeline

model_id = "CompVis/stable-diffusion-v1-4"
device = "cuda"
remove_safety = False


pipe = StableDiffusionPipeline.from_pretrained(model_id, scheduler=scheduler, torch_dtype=torch.float16, revision="fp16", use_auth_token=True)
if remove_safety:
  pipe.safety_checker = lambda images, clip_input: (images, False)
pipe = pipe.to(device)



Here is where you actually make images. Change the "prompt" to whatever you want to try and then change "num_images" if you want more than one image generated. You can re-run this cell without having to re-run everything before it, just FYI.

In [ ]:
prompt = "photo of Arbian Horses"
num_images = 2

prompts = [ prompt ] * num_images
with autocast("cuda"):
    images = pipe(prompts, guidance_scale=7.5, num_inference_steps=50).images

media.show_images(images)
images[0].save("output.jpg")

In [ ]:
import gradio as gr
import torch
from diffusers import StableDiffusionPipeline

def generate_image_with_diffusion(prompt):
    model_id = "CompVis/stable-diffusion-v1-4"
    device = "cuda"  # Change to "cpu" if you want to run on CPU
    remove_safety = False

    pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16, revision="fp16", use_auth_token=True)

    if remove_safety:
        pipe.safety_checker = lambda images, clip_input: (images, False)
    pipe = pipe.to(device)

    with torch.autocast("cuda"):
        images = pipe([prompt], guidance_scale=7.5, num_inference_steps=50).images

    return images[0]

# Gradio界面配置
inputs = gr.inputs.Textbox(label="输入文本")
outputs = gr.outputs.Image(label="生成的图像", type="pil")


# 创建Gradio界面
gr.Interface(
    fn=generate_image_with_diffusion,
    inputs=inputs,
    outputs=outputs,
    title="<span style='font-style: italic; font-weight: bold; color: darkred;'>Alec作品</span> -Stable Diffusion 图像艺术创作空间",
    description="详细输入您的想象，生成相应的图像"
).launch(share=True, debug=True)


#iface.launch(share=True, debug=True)
